In [1]:
from ngsolve.meshes import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import numpy as np
import time as timeit
from aux import *

%matplotlib notebook

ngsglobals.msg_level=1

def FVSolveBot(nx=200, order = 2, cfl=0.1, method ="RK3", 
               eps0 = 1e-16, eps1=1e-4, epsRho=1e-6,
               plot=False, tol = 0.04, printing=False, flag="comp",
              pcBot=False, limit=True, cpFlag = False):
    time = 0
    count = 0
    
    g = 9.812
    eps = (2/g)**0.5
    
    # the mesh
    mesh = MakeStructured2DMesh(quads=False, nx=nx, ny=nx, periodic_x=True,
                               periodic_y=True)
    h0 = 1/nx
    # bottom and data
    b0 = sin(2*pi*x)+cos(2*pi*y)
    rhoI = 10+exp(sin(2*pi*x))*cos(2*pi*y)
    uI = sin(cos(2*pi*x))*sin(2*pi*y)/rhoI
    vI = cos(2*pi*x)*cos(sin(2*pi*y))/rhoI
    rhobI = rhoI + b0
    tend = 0.05
    
    n = specialcf.normal(mesh.dim)
    # Burgers
    V = L2(mesh, order=order, all_dofs_together=False)
    fes = FESpace([V,V,V])

    # TVB limiter preparation
#     listNbr, listNbj, listNbw, alphaList, listNrms = getNeighbors2D(mesh)

    bh = GridFunction(V)
    # continuous Bottom
    tmpB = GridFunction(Periodic(H1(mesh,order=order)))
    rho0 = GridFunction(Periodic(H1(mesh,order=order)))
    
    gfu = GridFunction(fes)
    gfu0 = GridFunction(fes)
    
    rhoh, uh1, uh2 = gfu.components
    rhoh0, uh10, uh20 = gfu0.components
    rhoh1 = GridFunction(V)
    mh1 = GridFunction(V) # momentum
    mh2 = GridFunction(V) # momentum
    mh10 = GridFunction(V) # momentum
    mh20 = GridFunction(V) # momentum
    
    ### INITIAL DATA
    tmpB.Set(b0)
    bh.Set(tmpB)
    
    # initial data
    uh1.Set(uI)
    uh2.Set(vI)
    rho0.Set(rhobI)
    rhoh.Set(rho0) # THIS INTRODUCE NUMERICAL ERROR !    
    rhoh.vec.data -= bh.vec # remove heights
            
    # add mh
    mh1.vec.data = uh1.vec
    V.ApplyM(rho=rhoh, vec=mh1.vec)
    V.SolveM(rho=1, vec=mh1.vec)
    mh2.vec.data = uh2.vec
    V.ApplyM(rho=rhoh, vec=mh2.vec)
    V.SolveM(rho=1, vec=mh2.vec)
    
        
    (rho, u1, u2), (eta, v1, v2) = fes.TnT()
    rhoO, u1O, u2O = rho.Other(), u1.Other(), u2.Other()
    etaO, v1O, v2O = eta.Other(), v1.Other(), v2.Other()
    
    u = CF((u1,u2))
    v = CF((v1,v2))
    uO = CF((u1O,u2O))
    vO = CF((v1O,v2O))
    du = CF((grad(u1), grad(u2)), dims=(2,2))
    dv = CF((grad(v1), grad(v2)), dims=(2,2))
    
    # Hydro static reconstruction
    rhoL = rho
    rhoR = rhoO
    jmp_rho = rho-rhoO 
    avg_rho = 0.5*(rhoL+rhoR)+bh
    
    cL = (2*rhoL)**0.5/eps
    cR = (2*rhoR)**0.5/eps

    avg_rho = 0.5*(rho+rhoO)
    avg_u = 0.5*(u+uO)
    
    # Later!!!
    unL = u*n
    unR = uO*n
    abs_uL = IfPos(unL, unL, -unL)
    abs_uR = IfPos(unR, unR, -unR)
    
    speedL = cL+abs_uL
    speedR = cR+abs_uR

    ### ESTIMATE SPEED ?! FIXME LATER
    speed_rho = IfPos(speedL-speedR, speedL, speedR)    
    
    # EEC flux + stab
    flux_rho = 0.5*(rhoL*u+rhoR*uO)*n+0.5*speed_rho*jmp_rho
    # FIXME LATER
    flux_m = flux_rho*avg_u + 0.5*speed_rho*avg_rho*(u-uO)
        
    # Spatial operator (for rho updates)
    a = BilinearForm(fes, nonassemble=True)
    # volume contributions (skew symmetric)
    if order>0:
        a += ((-rho*u*grad(eta)
              +g*(grad(rho)+grad(bh))*rho*v
              -0.5*rho*dv*u*u+0.5*rho*du*u*v
              )*dx).Compile(cpFlag, True)
    # interior bdry contribution
    a += ((flux_rho*(eta-etaO)
          +flux_m*(v-vO)
           -0.5*flux_rho*(u*v-uO*vO)
          -g*jmp_rho*n*0.5*(rhoL*v+rhoR*vO)
          )*dx(skeleton=True)).Compile(cpFlag, True)
    
    tmp = gfu.vec.CreateVector()
   
    tmpU = uh1.vec.CreateVector()
    nrho = V.ndof

    step = 0
    
    # velocity estimation
    vmax = 10
    dt = cfl*h0/vmax
    
    def ForwardEuler(dt):
        # update height
        a.Apply(gfu.vec, tmp)
        # update derivative OLD part
#         tmpU.data = uh1.vec
#         V.ApplyM(rho=rhoh, vec=tmpU)
#         tmp[nrho:2*nrho].data += 0.5/dt*tmpU
#         tmpU.data = uh2.vec
#         V.ApplyM(rho=rhoh, vec=tmpU)
#         tmp[2*nrho:].data += 0.5/dt*tmpU
        
        # update density (unlimited)        
        V.SolveM(rho=1, vec=tmp[:nrho])
        rhoh.vec.data -= dt*tmp[:nrho]

        # update derivative NEW part
        tmpU.data = uh1.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[nrho:2*nrho].data -= 0.5/dt*tmpU

        tmpU.data = uh2.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[2*nrho:].data -= 0.5/dt*tmpU

        # update momentum (unlimited)
        V.SolveM(rho=1, vec=tmp[nrho:2*nrho])
        mh1.vec.data *= 0.5
        mh1.vec.data -= dt*tmp[nrho:2*nrho]

        # update momentum (unlimited)
        V.SolveM(rho=1, vec=tmp[2*nrho:])
        mh2.vec.data *= 0.5
        mh2.vec.data -= dt*tmp[2*nrho:]
            
    # convex combination
    def ConvexComb(w0, w1):
        # update density
        rhoh.vec.data *= w1
        rhoh.vec.data += w0*rhoh0.vec

        # update momentum
        mh1.vec.data *= w1
        mh1.vec.data += w0*mh10.vec

        # update momentum
        mh2.vec.data *= w1
        mh2.vec.data += w0*mh20.vec


    def CalcVel():
        tmpU.data = mh1.vec
        V.ApplyM(rho=1, vec=tmpU)
        V.SolveM(rho=rhoh, vec=tmpU)
        uh1.vec.data = tmpU
        
        tmpU.data = mh2.vec
        V.ApplyM(rho=1, vec=tmpU)
        V.SolveM(rho=rhoh, vec=tmpU)
        uh2.vec.data = tmpU

    
    nelems = mesh.ne
    
    vmax = 10
    with TaskManager():
        while time < tend-0.1*dt:
            time += dt
            if time > tend:
                time -= dt
                dt = tend-time
                time = tend
            step += 1
            gfu0.vec.data = gfu.vec
            mh10.vec.data = mh1.vec
            mh20.vec.data = mh2.vec

            ### RK3 stage 1
            ForwardEuler(dt)
            CalcVel()
            
            ForwardEuler(dt)
            ConvexComb(0.75, 0.25)
            CalcVel()
            
            ForwardEuler(dt)
            ConvexComb(1/3, 2/3)
            CalcVel()
            
            print("\r", format(step,'4.0f'), end="")
        print("\n")
        return rhoh, uh1, uh2, mh1, mh2


In [2]:
%%time
rhoh, uh1, uh2, mh1, mh2 = FVSolveBot(nx=50, order = 2, cfl=0.05, method ="RK3", cpFlag=False)
# Draw(rhoh, rhoh.space.mesh)

  500

CPU times: user 6min 30s, sys: 3.06 s, total: 6min 33s
Wall time: 6.28 s


In [3]:
%%time
## reference solution
rhoList, uList, mList = [], [], []
for i in range(4):
    nx = 25*2**i
    rhoh, uh1, uh2, mh1, mh2 = FVSolveBot(nx = nx, order=2, cfl = 0.05, method="RK3")
    rhoList.append(rhoh)
    uList.append([uh1, uh2])
    mList.append([mh1, mh2])

  250  41

  500

 1000

 2000

CPU times: user 5h 6min 7s, sys: 1min 19s, total: 5h 7min 26s
Wall time: 4min 50s


In [4]:
%%time 
# reference solution: takes roughly half an hour to get the result
rhoX, uh1X, uh2X, mh1X, mh2X = FVSolveBot(nx=400, order = 2, cfl=0.05, method ="RK3")

 4000

CPU times: user 1d 9h 4min 10s, sys: 7min 4s, total: 1d 9h 11min 15s
Wall time: 31min 15s


In [5]:
# calculation L2-errors
mesh = rhoX.space.mesh
with TaskManager():
    for i in range(4):
        errR = sqrt(Integrate((rhoList[i]-rhoX)**2, mesh))
        errU = sqrt(Integrate((uList[i][0]-uh1X)**2+(uList[i][1]-uh2X)**2, mesh))
        errM = sqrt(Integrate((mList[i][0]-mh1X)**2+(mList[i][1]-mh2X)**2, mesh))
        if i ==0:
            print("%3i& %.3e & -- & %.3e & -- & %.3e &-- \\\\"%(25*2**i, errR, errU, errM))
        else:
            rateR = -log(errR/errR0)/log(2)
            rateU = -log(errU/errU0)/log(2)
            rateM = -log(errM/errM0)/log(2)
            print("%3i& %.3e & %.2f & %.3e & %.2f & %.3e & %.2f \\\\"%(25*2**i, errR, rateR, errU, 
                                                                     rateU, errM, rateM))
        errR0 =errR
        errU0 = errU
        errM0 = errM    

 25& 1.420e-03 & -- & 1.670e-03 & -- & 1.379e-02 &-- \\
 50& 1.567e-04 & 3.18 & 2.083e-04 & 3.00 & 1.683e-03 & 3.03 \\
100& 1.917e-05 & 3.03 & 2.820e-05 & 2.89 & 2.371e-04 & 2.83 \\
200& 2.363e-06 & 3.02 & 3.880e-06 & 2.86 & 3.403e-05 & 2.80 \\
